In [ ]:
!pip install tensorflow==2.11

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
!sudo apt-get install tesseract-ocr

In [ ]:
!pip install pytesseract==0.3.9

In [ ]:
!pip install transformers

In [ ]:
!pip install pynvml

In [ ]:
!pip install wget

In [ ]:
!pip install git+https://github.com/openai/whisper.git

In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
!pip install youtube_dl

In [ ]:
!pip install kaleido

In [ ]:
!pip install deep-translator

In [13]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import h5py
import spacy
from textblob import TextBlob
import warnings
import nltk
from nltk.stem.wordnet import WordNetLemmatizer

warnings.filterwarnings("ignore")

In [ ]:
nltk.download("words")
nltk.download("wordnet")

In [15]:
import csv
import re
import requests
from datetime import timezone, datetime, timedelta
from dateutil import tz
import plotly
import plotly.graph_objects as go
import kaleido
from deep_translator import GoogleTranslator
from pynvml import *
import math
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import ConvBertModel, ConvBertTokenizer, TFConvBertModel

In [16]:
import tweepy
from tweepy import *
import wget
import cv2
import pytesseract
import os
import whisper
import time
import librosa
import youtube_dl

In [ ]:
model = whisper.load_model("small.en")  # load the small model for transcribing videos

In [18]:
nlp = spacy.load("en_core_web_lg")

In [48]:
# API key:
consumer_key = "blah"
# API key secret:
consumer_secret = "blah"
# Access token:
access_token = "blah"
# Access token secret:
access_token_secret = "blah"

# For Twitter API v1.1:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [20]:
end_date_utc = datetime(
    year=2023, month=3, day=4, hour=18, minute=59, second=59, tzinfo=timezone.utc
)
delta = timedelta(days=-60)
start_date_utc = end_date_utc + delta
end_date = end_date_utc.strftime("%m-%d-%Y")
start_date = start_date_utc.strftime("%m-%d-%Y")

In [ ]:
# To load ~9 politicians' accounts to analyze:
day_num = 1
accounts_df = pd.read_csv(
    "/content/drive/MyDrive/Colab_Notebooks/TwitterAccountList.csv"
)
polit_list_old = []
party_list = []
state_list = []
name_list = []
user_list_old = []
for i in range(len(accounts_df.loc[accounts_df["Grouping"] == day_num, "Usernames"])):
    user_list_old.append(
        accounts_df.loc[accounts_df["Grouping"] == day_num, "Usernames"].iloc[i]
    )
    party_list.append(
        accounts_df.loc[accounts_df["Grouping"] == day_num, "Party"].iloc[i]
    )
    state_list.append(
        accounts_df.loc[accounts_df["Grouping"] == day_num, "State"].iloc[i]
    )
    name_list.append(
        accounts_df.loc[accounts_df["Grouping"] == day_num, "Politician"].iloc[i]
    )
    polit_list_old.append(
        str(accounts_df.loc[accounts_df["Grouping"] == day_num, "AccountID_1"].iloc[i])
        + str(
            accounts_df.loc[accounts_df["Grouping"] == day_num, "AccountID_2"].iloc[i]
        )
    )
polit_list_old

In [22]:
# Optical Character Recognition (OCR) of text in photos
def process_pics(url, file_id):
    try:
        wget.download(url, out=file_id)
        img = cv2.imread(file_id)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        text = pytesseract.image_to_string(img)
        os.remove(file_id)
        return text
    except Exception as e:
        print(e)
        print(f"process_pics exception {url}")
        text = ""
        return text

In [23]:
# Use regex to replace \n with a space for tesseract OCR'ed text
def process_tess(input_text):
    pat_n1 = re.compile("(\\n)")
    pat_n2 = re.compile("(\\x0c)")
    text1 = re.sub(pat_n1, " ", input_text)
    return re.sub(pat_n2, "", text1)

In [24]:
# Transcribe from the audio of videos
def process_vids(url, file_id):
    ydl_opts = {"outtmpl": file_id, "quiet": True, "get-duration": True}
    try:
        # Adapted from https://www.bogotobogo.com/VideoStreaming/YouTube/youtube-dl-embedding.php
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        # Load the audio with 16 kHz sampling rate
        y, sr = librosa.load(file_id, sr=16000)
        result = model.transcribe(y)
        text = result["text"].strip()
        os.remove(file_id)
        return text
    except Exception as e:
        # print(e)
        print(type(e))
        print(f"process_vids exception {url} file {file_id}")
        return ""

In [25]:
# Updated with suggestions from ChatGPT
def transcr_media(df, jnum):
    path_id = "/content/drive/MyDrive/Colab_Notebooks/"
    pat_png = re.compile("\W(png)")
    transcr_vids = [[] for i in range(len(df))]
    transcr_pics = [[] for i in range(len(df))]
    transcr_vids_em = [[] for i in range(len(df))]
    transcr_pics_em = [[] for i in range(len(df))]
    for i in range(len(df)):
        if df.Videos[i] != []:
            for kk in range(len(df.Videos[i])):
                url = df.Videos[i][kk]
                file_id = path_id + jnum + "Tweet" + str(i) + str(kk) + ".mp4"
                transcr_vids[i].append(process_vids(url, file_id))
        if df.Pictures[i] != []:
            for kk in range(len(df.Pictures[i])):
                url = df.Pictures[i][kk]
                if bool(re.findall(pat_png, str(url))):
                    file_id = path_id + jnum + "Tweet" + str(i) + str(kk) + ".png"
                else:
                    file_id = path_id + jnum + "Tweet" + str(i) + str(kk) + ".jpg"
                transcr_pics[i].append(process_pics(url, file_id))
        if df.EmbedVideos[i] != []:
            for kk in range(len(df.Videos[i])):
                url = df.EmbedVideos[i][kk]
                file_id = path_id + jnum + "Embed" + str(i) + str(kk) + ".mp4"
                transcr_vids_em[i].append(process_vids(url, file_id))
        if df.EmbedPictures[i] != []:
            for kk in range(len(df.EmbedPictures[i])):
                url = df.EmbedPictures[i][kk]
                if bool(re.findall(pat_png, str(url))):
                    file_id = path_id + jnum + "Embed" + str(i) + str(kk) + ".png"
                else:
                    file_id = path_id + jnum + "Embed" + str(i) + str(kk) + ".jpg"
                transcr_pics_em[i].append(process_pics(url, file_id))
    df["Transcr_vids"] = transcr_vids
    df["Transcr_pics"] = transcr_pics
    df["Transcr_vids_em"] = transcr_vids_em
    df["Transcr_pics_em"] = transcr_pics_em
    return df

In [26]:
# Adapted from ChatGPT
# Assume average word length is 8 characters to stay below translate character limit of 5000
def batch_translate(text, batch_len=600):
    pat_text = re.compile(r"[A-Za-z]{1,50}")
    if not re.findall(pat_text, text):
        translated_text = ""
    else:
        # Split the text into batches of words
        word_batches = []
        wordies = text.split()
        num_batches = math.ceil(len(wordies) / batch_len)
        for i in range(num_batches):
            word_batch = ""
            start_index = i * batch_len
            end_index = (i + 1) * batch_len
            if end_index > len(wordies):
                end_index = len(wordies)
            for im in range(start_index, end_index):
                word_batch += wordies[im] + " "
            word_batches.append(word_batch)

        # Translate each batch and combine the results
        translated_text = ""
        for batch in word_batches:
            translated_batch = GoogleTranslator(source="auto", target="en").translate(
                batch
            )
            translated_text += translated_batch + " "

    return translated_text.strip()

In [27]:
# Use regex to clean up some of the transcriptions; translate to English
# Adapted from: https://prrao87.github.io/blog/spacy/nlp/performance/2020/05/02/spacy-multiprocess.html
def transcr_clean(df):
    pattern = re.compile(r"[A-Za-z0-9\-\.]{2,50}")
    # Regex pattern for only alphanumeric, hyphenated text with 2 or more characters; removes punctuation other than period
    df["PreCleanVids"] = df["Transcr_vids"].copy()
    df["PreCleanPics"] = df["Transcr_pics"].copy()
    for i in df.index:
        for kk in range(len(df.Transcr_vids[i])):
            df["PreCleanVids"][i][kk] = " ".join(
                re.findall(pattern, batch_translate(str(df["Transcr_vids"][i][kk])))
            )
        for kk in range(len(df.Transcr_pics[i])):
            df["PreCleanPics"][i][kk] = " ".join(
                re.findall(pattern, batch_translate(str(df["Transcr_pics"][i][kk])))
            )

    df["PreCleanVids_em"] = df["Transcr_vids_em"].copy()
    df["PreCleanPics_em"] = df["Transcr_pics_em"].copy()
    for i in df.index:
        for kk in range(len(df.Transcr_vids_em[i])):
            df["PreCleanVids_em"][i][kk] = " ".join(
                re.findall(pattern, batch_translate(str(df["Transcr_vids_em"][i][kk])))
            )
        for kk in range(len(df.Transcr_pics_em[i])):
            df["PreCleanPics_em"][i][kk] = " ".join(
                re.findall(pattern, batch_translate(str(df["Transcr_pics_em"][i][kk])))
            )

    return df

In [28]:
# Made more efficient with ChatGPT
def filter_non_words(df):
    transcr_pics_clean, transcr_pics_clean_em = [], []
    # Adapted from https://intellipaat.com/community/5638/removing-non-english-words-from-text-using-python
    # Used both spacy and nltk lemmatizers because a word like "governors" was included this way
    # whereas it wasn't when only the nltk lemmatizer was used
    # splitting into batches of words make this faster because the spacy-identified entities list is shorter
    # and don't have to test the whole list of words against them, only the batch of words
    words = set(nltk.corpus.words.words())
    Lem = WordNetLemmatizer()
    batch_len = 10
    for i in df.index:
        transcr_pics_clean.append([])
        transcr_pics_clean_em.append([])
        if df.PreCleanPics[i] != []:
            for kj in range(len(df.PreCleanPics[i])):
                clean_pics = ""
                # Split the text into batches
                word_batches = []
                wordies = str(df.PreCleanPics[i][kj]).split()
                num_batches = math.ceil(len(wordies) / batch_len)
                for imi in range(num_batches):
                    word_batch = ""
                    start_index = imi * batch_len
                    end_index = (imi + 1) * batch_len
                    if end_index > len(wordies):
                        end_index = len(wordies)
                    for im in range(start_index, end_index):
                        word_batch += wordies[im] + " "
                    word_batches.append(word_batch)
                for batch in word_batches:
                    doc = nlp(batch.lower())
                    for token in doc:
                        lemword = Lem.lemmatize(token.text)
                        if (
                            lemword.lower() in words
                            or token.lemma_ in words
                            or not lemword.isalpha()
                        ):
                            if token.text == ".":
                                clean_pics = clean_pics.strip()
                            clean_pics += token.text.lower() + " "
                            continue
                        else:
                            for ent in doc.ents:
                                doc_ent = nlp(ent.text)
                                for tok_ent in doc_ent:
                                    if lemword.lower() == tok_ent.text:
                                        clean_pics += token.text.lower() + " "
                                        break
                transcr_pics_clean[i].append(clean_pics.strip())
        if df.PreCleanPics_em[i] != []:
            for kj in range(len(df.PreCleanPics_em[i])):
                clean_pics_em = ""
                # Split the text into batches
                word_batches = []
                wordies = str(df.PreCleanPics_em[i][kj]).split()
                num_batches = math.ceil(len(wordies) / batch_len)
                for imi in range(num_batches):
                    word_batch = ""
                    start_index = imi * batch_len
                    end_index = (imi + 1) * batch_len
                    if end_index > len(wordies):
                        end_index = len(wordies)
                    for im in range(start_index, end_index):
                        word_batch += wordies[im] + " "
                    word_batches.append(word_batch)
                for batch in word_batches:
                    doc_em = nlp(batch.lower())
                    for token in doc_em:
                        lemword = Lem.lemmatize(token.text)
                        if (
                            lemword.lower() in words
                            or token.lemma_ in words
                            or not lemword.isalpha()
                        ):
                            if token.text == ".":
                                clean_pics_em = clean_pics_em.strip()
                            clean_pics_em += token.text.lower() + " "
                            continue
                        else:
                            for ent in doc_em.ents:
                                doc_ent_em = nlp(ent.text)
                                for tok_ent in doc_ent_em:
                                    if lemword.lower() == tok_ent.text:
                                        clean_pics_em += token.text.lower() + " "
                                        break
                transcr_pics_clean_em[i].append(clean_pics_em.strip())
    df["Clean_pics"] = transcr_pics_clean
    df["Clean_pics_em"] = transcr_pics_clean_em
    return df

In [29]:
# If they're short (<140 characters), add media transcriptions to the text of the tweet,
# otherwise add them to the text of "SpeechToRate"
def sort_transcripts(df):
    vid_count, pic_count = 0, 0
    tweet_rate, speech_rate, embed_rate = [], [], []
    for i in df.index:
        tweet_rate.append(str(df.Translated[i]))
        embed_rate.append(str(df.EmbedTrans[i]))
        speech_rate.append("")
        for kk in range(len(df.Clean_pics[i])):
            if str(df.Clean_pics[i][kk]) != "nan":
                pic_count += 1
                if len(str(df.Clean_pics[i][kk])) < 140:
                    tweet_rate[i] += " " + str(df.Clean_pics[i][kk])
                else:
                    speech_rate[i] += str(df.Clean_pics[i][kk]) + " "
        for kk in range(len(df.PreCleanVids[i])):
            if str(df.PreCleanVids[i][kk]) != "nan":
                vid_count += 1
                if len(str(df.PreCleanVids[i][kk])) < 140:
                    tweet_rate[i] += " " + str(df.PreCleanVids[i][kk])
                else:
                    speech_rate[i] += str(df.PreCleanVids[i][kk]) + " "
        speech_rate[i].strip()

        for kk in range(len(df.Clean_pics_em[i])):
            if str(df.Clean_pics_em[i][kk]) != "nan":
                pic_count += 1
                if len(str(df.Clean_pics_em[i][kk])) < 140:
                    embed_rate[i] += " " + str(df.Clean_pics_em[i][kk])
                else:
                    speech_rate[i] += str(df.Clean_pics_em[i][kk]) + " "
        for kk in range(len(df.PreCleanVids_em[i])):
            if str(df.PreCleanVids_em[i][kk]) != "nan":
                vid_count += 1
                if len(str(df.PreCleanVids_em[i][kk])) < 140:
                    embed_rate[i] += " " + str(df.PreCleanVids_em[i][kk])
                else:
                    speech_rate[i] += str(df.PreCleanVids_em[i][kk]) + " "
        speech_rate[i].strip()

    df["EmbedToRate"] = embed_rate
    df["TweetToRate"] = tweet_rate
    df["SpeechToRate"] = speech_rate
    return df, pic_count, vid_count

In [30]:
# ChatGPT suggested improved version:
def process_users(input_text: str) -> str:
    pat_user = re.compile("@([A-Za-z\d_])\w+")
    if not input_text:
        return ""
    return pat_user.sub("name", input_text)

In [31]:
# ChatGPT-improved version
# Expand select abbreviations (such as imo, RT, and lol), before expanding hashtags
def process_abbrevs(input_text: str) -> str:
    pat_abbrev_dict = {
        re.compile(r"\bimo\b"): "in my opinion",
        re.compile(r"\bRT\b"): "retweet",
        re.compile(r"\bimho\b"): "in my humble opinion",
        re.compile(r"\blol\b"): "laugh out loud",
        re.compile(r"\blmao\b"): "laugh my ass off",
        re.compile(r"\bgov't\b"): "government",
        re.compile(r"\bBidenflation\b"): "biden's inflation",
        re.compile(r"\bBidenomics\b"): "biden's economic policies",
        re.compile(r"\bmaga\b"): "make america great again",
        re.compile(r"\bACA\b"): "affordable care act",
        re.compile(r"\bObamacare\b"): "affordable care act",
        re.compile(r"\bImo\b"): "in my opinion",
        re.compile(r"\bIMO\b"): "in my opinion",
        re.compile(r"\bImho\b"): "in my humble opinion",
        re.compile(r"\bIMHO\b"): "in my humble opinion",
        re.compile(r"\bMAGA\b"): "make america great again",
        re.compile(r"\bLol\b"): "laugh out loud",
        re.compile(r"\bLOL\b"): "laugh out loud",
        re.compile(r"\bLmao\b"): "laugh my ass off",
        re.compile(r"\bLMAO\b"): "laugh my ass off",
    }
    if not input_text:
        return ""
    for i in pat_abbrev_dict.keys():
        output_text = re.sub(i, pat_abbrev_dict[i], input_text)
    return output_text

In [32]:
# Breaking up hashtags into individual words by capitalization, e.g. #NeedMoreMoney = "need more money"
# Adapted from https://stackoverflow.com/questions/68448243/efficient-way-to-split-multi-word-hashtag-in-python
def process_hashtags(input_text: str) -> str:
    return re.sub(
        r"#[A-Za-z]\S*",
        lambda m: " ".join(
            re.findall("[A-Z][^A-Z]*|[a-z][^A-Z]*", m.group().lstrip("#"))
        ),
        input_text,
    )

In [33]:
# Replace urls with "weblink" and make everything lowercase
# Adapted from https://stackoverflow.com/questions/839994/extracting-a-url-in-python
def process_urls(input_text: str) -> str:
    pat_urls = re.compile(r"(https?:\/\/\S+\b)")
    if not input_text:
        return ""
    output_text = re.sub(pat_urls, "weblink", input_text)
    output_text = output_text.lower()
    return output_text

In [34]:
# Adapted from: https://prrao87.github.io/blog/spacy/nlp/performance/2020/05/02/spacy-multiprocess.html
# Regex pattern for tweet text: only alphanumeric, hyphenated text with 2 or more characters; removes other punctuation
def cleaner(df):
    pat_clean = re.compile(r"[A-Za-z0-9\-]{2,50}")
    df["cleaned_tweet"] = df["preclean"].str.findall(pat_clean).str.join(" ")
    return df

In [35]:
# Function to break text into sentences using spacy
def sentencize(text):
    doc = nlp(text)
    doc_sents = [str(sent) for sent in doc.sents]
    return doc_sents

In [36]:
# Function to make a list of all the sentences, in order
def sentencelist(df):
    df_sents_all = []
    for i in df.index:
        for jj in range(len(df.sentences[i])):
            if str(df.sentences.at[i][jj]) != "":
                df_sents_all.append(str(df.sentences.at[i][jj]))
    return df_sents_all

In [37]:
# sentencelistize - break transcript into sentences and list in a dataframe
def sentencelistize(df):
    df["sentences"] = df["SpeechToRate"].apply(sentencize)
    df_sents = sentencelist(df)
    df_new = pd.DataFrame(pd.Series(df_sents), columns=["sentences"])
    return df_new

In [38]:
# Group the transcript sentences into "sections" and write to a dataframe for input into ratings NN
# Going from a dataframe with "sentences" to one with just "sections," the size of tweets
def sectionize_astweets(df):
    current_section = ""
    current_length = 0
    sections = []
    max_sent_len = df.sentences.str.len().max()
    if max_sent_len >= 280:
        for sentence in df["sentences"]:
            if current_length > 0 and current_length + len(sentence) >= 280:
                sections.append(current_section)
                current_section = sentence
                current_length = len(sentence)
            else:
                current_section += sentence + " "
                current_length += len(sentence) + 1
    else:
        for sentence in df["sentences"]:
            if current_length + len(sentence) >= 280:
                sections.append(current_section)
                current_section = sentence
                current_length = len(sentence)
            else:
                current_section += sentence + " "
                current_length += len(sentence) + 1
    if current_length != 0:
        sections.append(current_section)

    df_new_sect = pd.DataFrame({"sections": sections})

    return df_new_sect, max_sent_len

In [39]:
# This is for processing one "SpeechToRate" row from X_speech dataframe,
# by breaking it up into 2 "tweet-size" sections and rating using
# models built from tweet text
def rate_speech_astweet(string):
    dict_string = {"SpeechToRate": string}
    df = pd.DataFrame(dict_string, index=[0])
    df_new = sentencelistize(df)
    df_new_sect, max_sent_len = sectionize_astweets(df_new)
    sectioned_text = ""
    # This is for outputting so can see where sectioned the sentences:
    for ii in df_new_sect.index:
        sectioned_text += df_new_sect.sections[ii] + " !@@! "
    sectioned_text += str(max_sent_len)
    # Predict ratings on tweet video and picture transcripts
    nb_batches = math.ceil(len(df_new_sect) / BATCH_SIZE)
    y_pred_m, y_pred_e, y_pred_n, y_pred_b = 0.0, 0.0, 0.0, 0.0
    for i in range(nb_batches):
        input_texts = df_new_sect[i * BATCH_SIZE : (i + 1) * BATCH_SIZE][
            "sections"
        ].tolist()
        encoded = tokenizer(
            input_texts,
            truncation=True,
            add_special_tokens=True,
            padding="max_length",
            max_length=80,
            return_tensors="tf",
            return_token_type_ids=False,
            return_attention_mask=True,
            verbose=True,
        )
        pred_raw_m = misrep_model.predict(
            {
                "input_ids": encoded["input_ids"],
                "attention_mask": encoded["attention_mask"],
            },
            verbose=0,
        )
        y_preds_m = np.argmax(pred_raw_m, axis=1)
        for kl in range(len(y_preds_m)):
            y_pred_m += decoded_dict_misrep[y_preds_m[kl]]
        pred_raw_e = entitle_model.predict(
            {
                "input_ids": encoded["input_ids"],
                "attention_mask": encoded["attention_mask"],
            },
            verbose=0,
        )
        y_preds_e = np.argmax(pred_raw_e, axis=1)
        for kl in range(len(y_preds_e)):
            y_pred_e += decoded_dict_entitle[y_preds_e[kl]]
        pred_raw_n = neg_model.predict(
            {
                "input_ids": encoded["input_ids"],
                "attention_mask": encoded["attention_mask"],
            },
            verbose=0,
        )
        y_preds_n = np.argmax(pred_raw_n, axis=1)
        for kl in range(len(y_preds_n)):
            y_pred_n += decoded_dict_neg[y_preds_n[kl]]
        pred_raw_b = blame_model.predict(
            {
                "input_ids": encoded["input_ids"],
                "attention_mask": encoded["attention_mask"],
            },
            verbose=0,
        )
        y_preds_b = np.argmax(pred_raw_b, axis=1)
        for kl in range(len(y_preds_b)):
            y_pred_b += decoded_dict_blame[y_preds_b[kl]]
    subject_blob_speech, polar_blob_speech, polar_blob_pos_speech = 0.0, 0.0, 0.0
    for mm in range(len(df_new)):
        subject_blob_speech += getSubjectivity(df_new.sentences[mm])
        if getPolarity(df_new.sentences[mm]) < 0:
            polar_blob_speech += getPolarity(df_new.sentences[mm])
        else:
            polar_blob_pos_speech += getPolarity(df_new.sentences[mm])
    return (
        y_pred_m,
        y_pred_e,
        y_pred_n,
        y_pred_b,
        subject_blob_speech,
        polar_blob_speech,
        polar_blob_pos_speech,
        sectioned_text,
    )

In [40]:
# Adapted from https://www.topcoder.com/thrive/articles/getting-started-with-textblob-for-sentiment-analysis#:~:text=When%20a%20sentence%20is%20passed,to%20personal%20opinions%20and%20judgments.
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity


def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [41]:
# Replace @Usernames, abbreviations, #HashTags, and https://weblinks in tweets, translate to English, and Regex clean
def preprocess_tweets(df):
    df["preclean"] = df["Tweet"].apply(process_abbrevs)
    df["preclean"] = df["preclean"].apply(process_hashtags)
    df["preclean"] = df["preclean"].apply(process_urls)
    df["preclean"] = df["preclean"].apply(process_users)
    df = cleaner(df)
    df = df.drop(columns=["Tweet", "preclean"], axis=1)
    df["polarity"] = df["cleaned_tweet"].apply(getPolarity)
    df["subjectivity"] = df["cleaned_tweet"].apply(getSubjectivity)
    return df

In [42]:
# Dictionaries to decode the tweet NN outputs back to actual ratings values in "years"
decoded_dict_neg = {0: 0, 1: 2, 2: 3, 3: 5, 4: 7, 5: 11.1}
decoded_dict_misrep = {0: 0, 1: 1, 2: 2, 3: 3.2}
decoded_dict_entitle = {0: 0, 1: 2, 2: 5}
decoded_dict_blame = {0: 0, 1: 2}

In [43]:
def rate_tweets(df):
    nb_batches = math.ceil(len(df) / BATCH_SIZE)
    y_pred_m, y_pred_e, y_pred_n, y_pred_b = 0.0, 0.0, 0.0, 0.0
    y_pred_ml, y_pred_el, y_pred_nl, y_pred_bl = [], [], [], []
    for i in range(nb_batches):
        input_texts = df[i * BATCH_SIZE : (i + 1) * BATCH_SIZE][
            "cleaned_tweet"
        ].tolist()
        encoded = tokenizer(
            input_texts,
            truncation=True,
            add_special_tokens=True,
            padding="max_length",
            max_length=80,
            return_tensors="tf",
            return_token_type_ids=False,
            return_attention_mask=True,
            verbose=True,
        )
        pred_raw_m = misrep_model.predict(
            {
                "input_ids": encoded["input_ids"],
                "attention_mask": encoded["attention_mask"],
            },
            verbose=0,
        )
        y_preds_m = np.argmax(pred_raw_m, axis=1)
        for kl in range(len(y_preds_m)):
            y_pred_m += decoded_dict_misrep[y_preds_m[kl]]
            y_pred_ml.append(decoded_dict_misrep[y_preds_m[kl]])
        pred_raw_e = entitle_model.predict(
            {
                "input_ids": encoded["input_ids"],
                "attention_mask": encoded["attention_mask"],
            },
            verbose=0,
        )
        y_preds_e = np.argmax(pred_raw_e, axis=1)
        for kl in range(len(y_preds_e)):
            y_pred_e += decoded_dict_entitle[y_preds_e[kl]]
            y_pred_el.append(decoded_dict_entitle[y_preds_e[kl]])
        pred_raw_n = neg_model.predict(
            {
                "input_ids": encoded["input_ids"],
                "attention_mask": encoded["attention_mask"],
            },
            verbose=0,
        )
        y_preds_n = np.argmax(pred_raw_n, axis=1)
        for kl in range(len(y_preds_n)):
            y_pred_n += decoded_dict_neg[y_preds_n[kl]]
            y_pred_nl.append(decoded_dict_neg[y_preds_n[kl]])
        pred_raw_b = blame_model.predict(
            {
                "input_ids": encoded["input_ids"],
                "attention_mask": encoded["attention_mask"],
            },
            verbose=0,
        )
        y_preds_b = np.argmax(pred_raw_b, axis=1)
        for kl in range(len(y_preds_b)):
            y_pred_b += decoded_dict_blame[y_preds_b[kl]]
            y_pred_bl.append(decoded_dict_blame[y_preds_b[kl]])
    return (
        y_pred_ml,
        y_pred_el,
        y_pred_nl,
        y_pred_bl,
        y_pred_m,
        y_pred_e,
        y_pred_n,
        y_pred_b,
    )

In [44]:
def rate_embeds(df):
    y_pred_m, y_pred_e, y_pred_n, y_pred_b = 0.0, 0.0, 0.0, 0.0
    y_pred_ml, y_pred_el, y_pred_nl, y_pred_bl = [], [], [], []
    for i in range(len(df)):
        if df.cleaned_tweet[i] != "":
            input_texts = df.cleaned_tweet[i]
            encoded = tokenizer(
                input_texts,
                truncation=True,
                add_special_tokens=True,
                padding="max_length",
                max_length=80,
                return_tensors="tf",
                return_token_type_ids=False,
                return_attention_mask=True,
                verbose=True,
            )
            pred_raw_m = misrep_model.predict(
                {
                    "input_ids": encoded["input_ids"],
                    "attention_mask": encoded["attention_mask"],
                },
                verbose=0,
            )
            y_preds_m = np.argmax(pred_raw_m, axis=1)
            for kl in range(len(y_preds_m)):
                y_pred_m += decoded_dict_misrep[y_preds_m[kl]]
                y_pred_ml.append(decoded_dict_misrep[y_preds_m[kl]])
            pred_raw_e = entitle_model.predict(
                {
                    "input_ids": encoded["input_ids"],
                    "attention_mask": encoded["attention_mask"],
                },
                verbose=0,
            )
            y_preds_e = np.argmax(pred_raw_e, axis=1)
            for kl in range(len(y_preds_e)):
                y_pred_e += decoded_dict_entitle[y_preds_e[kl]]
                y_pred_el.append(decoded_dict_entitle[y_preds_e[kl]])
            pred_raw_n = neg_model.predict(
                {
                    "input_ids": encoded["input_ids"],
                    "attention_mask": encoded["attention_mask"],
                },
                verbose=0,
            )
            y_preds_n = np.argmax(pred_raw_n, axis=1)
            for kl in range(len(y_preds_n)):
                y_pred_n += decoded_dict_neg[y_preds_n[kl]]
                y_pred_nl.append(decoded_dict_neg[y_preds_n[kl]])
            pred_raw_b = blame_model.predict(
                {
                    "input_ids": encoded["input_ids"],
                    "attention_mask": encoded["attention_mask"],
                },
                verbose=0,
            )
            y_preds_b = np.argmax(pred_raw_b, axis=1)
            for kl in range(len(y_preds_b)):
                y_pred_b += decoded_dict_blame[y_preds_b[kl]]
                y_pred_bl.append(decoded_dict_blame[y_preds_b[kl]])
        else:
            y_pred_ml.append(0)
            y_pred_el.append(0)
            y_pred_nl.append(0)
            y_pred_bl.append(0)
    return (
        y_pred_ml,
        y_pred_el,
        y_pred_nl,
        y_pred_bl,
        y_pred_m,
        y_pred_e,
        y_pred_n,
        y_pred_b,
    )

In [45]:
# Load ConvBERT tweet ratings models (multiclass classification)
entitle_model = load_model(
    "/content/drive/MyDrive/Colab_Notebooks/model_ConvBert_tweets_entitlev23PMmulticlass.h5",
    custom_objects={"TFConvBertModel": TFConvBertModel},
)
misrep_model = load_model(
    "/content/drive/MyDrive/Colab_Notebooks/model_ConvBert_tweets_misrepv17PMmulticlass.h5",
    custom_objects={"TFConvBertModel": TFConvBertModel},
)
neg_model = load_model(
    "/content/drive/MyDrive/Colab_Notebooks/model_ConvBert_tweets_negv17PMmulticlass.h5",
    custom_objects={"TFConvBertModel": TFConvBertModel},
)
blame_model = load_model(
    "/content/drive/MyDrive/Colab_Notebooks/model_ConvBert_tweets_blamev17PMmulticlass.h5",
    custom_objects={"TFConvBertModel": TFConvBertModel},
)

In [ ]:
v_num = "17b"
BASE_MODEL = "YituTech/conv-bert-base"
BATCH_SIZE = 32
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

In [ ]:
# Read in tweets for processing to ratings in units of lifetimes
# Guide to variables: "_t" indicates for tweets, "_em" for embedded tweets and retweets, and
# "_s" for "speech," or transcripts of videos and photos
polit_list = polit_list_old
user_list_start = user_list_old
# The following variables are for saving ratings for each member of polit_list:
user_list = []
polit_list_new = []
misrep_pred_life_t, entitle_pred_life_t, neg_pred_life_t = [], [], []
blame_pred_life_t = []
misrep_pred_life_em, entitle_pred_life_em, neg_pred_life_em = [], [], []
blame_pred_life_em = []
misrep_pred_life_s, entitle_pred_life_s, neg_pred_life_s = [], [], []
blame_pred_life_s = []
misrep_pred_life_tot, entitle_pred_life_tot, neg_pred_life_tot = [], [], []
blame_pred_life_tot = []
subjectivity_blob_t, polarity_blob_t, polarity_blob_pos_t = [], [], []
subjectivity_blob_em, polarity_blob_em, polarity_blob_pos_em = [], [], []
subjectivity_blob_s, polarity_blob_s, polarity_blob_pos_s = [], [], []
num_tweets, pics_count, vids_count = [], [], []
k = -1
for ki, j in enumerate(polit_list):
    print(j)
    try:
        user = api.get_user(user_id=j)
        j_name = user.screen_name
    except Exception as ex:
        print(ex)
        j_name = user_list_start[ki]
    try:
        k += 1
        j_prev_name = user_list_start[k]
        print(j_name)
        tweets_df = pd.read_pickle(
            "/content/drive/MyDrive/Colab_Notebooks/PolitTweets/TwoMonthTweets"
            + end_date
            + "/"
            + j_prev_name
            + "TwoMonthTweets"
            + str(j)
            + "end"
            + end_date
            + ".pkl"
        )
    except Exception as e:
        print(e)
        continue
    if len(tweets_df) == 0:
        continue
    user_list.append(j_name)
    polit_list_new.append(j)
    num_tweets.append(len(tweets_df))
    tweets_df = transcr_media(tweets_df, j)
    tweets_df = transcr_clean(tweets_df)
    tweets_df = filter_non_words(tweets_df)
    tweets_df, pics_counts, vids_counts = sort_transcripts(tweets_df)
    pics_count.append(pics_counts)
    vids_count.append(vids_counts)
    X_speech = pd.DataFrame().assign(SpeechToRate=tweets_df["SpeechToRate"])
    X_tweets = pd.DataFrame().assign(Tweet=tweets_df["TweetToRate"])
    X_embeds = pd.DataFrame().assign(Tweet=tweets_df["EmbedToRate"])
    # Rate each row of X_speech and add them up:
    misrep_pred_speech, entitle_pred_speech, neg_pred_speech = 0.0, 0.0, 0.0
    blame_pred_speech = 0.0
    subject_blob_speech, polar_blob_speech, polar_blob_pos_speech = 0.0, 0.0, 0.0
    # These variables are for saving ratings for each individual "speech" (transcribed videos and text from photos):
    misrep_pred_s, entitle_pred_s, neg_pred_s = [], [], []
    blame_pred_s, sectioned_text = [], []
    subject_blob_s, polar_blob_s, polar_blob_pos_s = [], [], []
    for m in X_speech.index:
        misrep_pred_s.append(0.0)
        entitle_pred_s.append(0.0)
        neg_pred_s.append(0.0)
        blame_pred_s.append(0.0)
        subject_blob_s.append(0.0)
        polar_blob_s.append(0.0)
        polar_blob_pos_s.append(0.0)
        sectioned_text.append("")
        if X_speech.SpeechToRate[m] != "":
            (
                misrep_pred_s[m],
                entitle_pred_s[m],
                neg_pred_s[m],
                blame_pred_s[m],
                subject_blob_s[m],
                polar_blob_s[m],
                polar_blob_pos_s[m],
                sectioned_text[m],
            ) = rate_speech_astweet(X_speech.SpeechToRate[m])
            misrep_pred_speech += misrep_pred_s[m]
            entitle_pred_speech += entitle_pred_s[m]
            neg_pred_speech += neg_pred_s[m]
            blame_pred_speech += blame_pred_s[m]
            # don't differentiate retweet/embedded tweet in textblob ratings
            subject_blob_speech += subject_blob_s[m]
            polar_blob_speech += polar_blob_s[m]
            polar_blob_pos_speech += polar_blob_pos_s[m]
    for i in X_tweets.index:
        X_tweets.Tweet[i] = str(X_tweets.Tweet[i])
    X_tweets = preprocess_tweets(X_tweets)
    for i in X_embeds.index:
        X_embeds.Tweet[i] = str(X_embeds.Tweet[i])
    X_embeds = preprocess_tweets(X_embeds)
    tweets_df["cleaned_tweet"] = X_tweets.cleaned_tweet.copy()
    tweets_df["cleaned_embed"] = X_embeds.cleaned_tweet.copy()
    tweets_df.to_csv(
        "/content/drive/MyDrive/Colab_Notebooks/PolitTweets/TwoMonthTranscribes"
        + end_date
        + "/TwoMonthTranscribedv"
        + v_num
        + j_name
        + end_date
        + "user"
        + str(j)
        + "V2.csv",
        index=False,
    )
    tweets_df.to_pickle(
        "/content/drive/MyDrive/Colab_Notebooks/PolitTweets/TwoMonthTranscribes"
        + end_date
        + "/TwoMonthTranscribedv"
        + v_num
        + j_name
        + end_date
        + "user"
        + str(j)
        + "V2.pkl"
    )
    (
        misrep_pred_t,
        entitle_pred_t,
        neg_pred_t,
        blame_pred_t,
        misrep_pred_tweet,
        entitle_pred_tweet,
        neg_pred_tweet,
        blame_pred_tweet,
    ) = rate_tweets(X_tweets)
    (
        misrep_pred_em,
        entitle_pred_em,
        neg_pred_em,
        blame_pred_em,
        misrep_pred_embed,
        entitle_pred_embed,
        neg_pred_embed,
        blame_pred_embed,
    ) = rate_embeds(X_embeds)
    misrep_pred_retweet, entitle_pred_retweet, neg_pred_retweet = 0.0, 0.0, 0.0
    blame_pred_retweet = 0.0
    for i in range(len(tweets_df)):
        if tweets_df.Retweet[i] == 1:
            misrep_pred_retweet += misrep_pred_t[i]
            entitle_pred_retweet += entitle_pred_t[i]
            neg_pred_retweet += neg_pred_t[i]
            blame_pred_retweet += blame_pred_t[i]
    # Put ratings of retweets into the embedded tweets rating, and take out of the regular tweet ratings
    misrep_pred_tweet -= misrep_pred_retweet
    misrep_pred_embed += misrep_pred_retweet
    entitle_pred_tweet -= entitle_pred_retweet
    entitle_pred_embed += entitle_pred_retweet
    neg_pred_tweet -= neg_pred_retweet
    neg_pred_embed += neg_pred_retweet
    blame_pred_tweet -= blame_pred_retweet
    blame_pred_embed += blame_pred_retweet
    misrep_pred_life_t.append(round((misrep_pred_tweet) / 60.0, 2))
    entitle_pred_life_t.append(round((entitle_pred_tweet) / 60.0, 2))
    neg_pred_life_t.append(round((neg_pred_tweet) / 60.0, 2))
    blame_pred_life_t.append(round((blame_pred_tweet) / 60.0, 2))
    misrep_pred_life_em.append(round((misrep_pred_embed) / 60.0, 2))
    entitle_pred_life_em.append(round((entitle_pred_embed) / 60.0, 2))
    neg_pred_life_em.append(round((neg_pred_embed) / 60.0, 2))
    blame_pred_life_em.append(round((blame_pred_embed) / 60.0, 2))
    subject_blob_tweet, polar_blob_tweet, polar_blob_pos_tweet = 0.0, 0.0, 0.0
    subject_blob_t, polar_blob_t, polar_blob_pos_t = [], [], []
    subject_blob_embed, polar_blob_embed, polar_blob_pos_embed = 0.0, 0.0, 0.0
    subject_blob_em, polar_blob_em, polar_blob_pos_em = [], [], []
    for i in range(len(X_tweets)):
        polar_blob_t.append(0.0)
        polar_blob_em.append(0.0)
        polar_blob_pos_t.append(0.0)
        polar_blob_pos_em.append(0.0)
        if X_tweets.polarity[i] < 0:
            polar_blob_tweet += X_tweets.polarity[i]
            polar_blob_t[i] = X_tweets.polarity[i]
        else:
            polar_blob_pos_tweet += X_tweets.polarity[i]
            polar_blob_pos_t[i] = X_tweets.polarity[i]
        if X_embeds.polarity[i] < 0:
            polar_blob_embed += X_embeds.polarity[i]
            polar_blob_em[i] = X_embeds.polarity[i]
        else:
            polar_blob_pos_embed += X_embeds.polarity[i]
            polar_blob_pos_em[i] = X_embeds.polarity[i]
    subject_blob_tweet = X_tweets.subjectivity.sum()
    subject_blob_embed = X_embeds.subjectivity.sum()
    subject_blob_t = X_tweets["subjectivity"].values.tolist()
    subject_blob_em = X_embeds["subjectivity"].values.tolist()
    # Save the ratings converted from units of years to lifetimes, for each member of polit_list
    misrep_pred_life_s.append(misrep_pred_speech / 60.0)
    entitle_pred_life_s.append(entitle_pred_speech / 60.0)
    neg_pred_life_s.append(neg_pred_speech / 60.0)
    blame_pred_life_s.append(blame_pred_speech / 60.0)
    subjectivity_blob_t.append(subject_blob_tweet)
    polarity_blob_t.append(polar_blob_tweet)
    polarity_blob_pos_t.append(polar_blob_pos_tweet)
    subjectivity_blob_em.append(subject_blob_tweet)
    polarity_blob_em.append(polar_blob_tweet)
    polarity_blob_pos_em.append(polar_blob_pos_tweet)
    subjectivity_blob_s.append(subject_blob_speech)
    polarity_blob_s.append(polar_blob_speech)
    polarity_blob_pos_s.append(polar_blob_pos_speech)

    tweets_rated = tweets_df.copy()
    tweets_rated["SectionedSpeech"] = sectioned_text
    tweets_rated["Misrep_t"] = misrep_pred_t
    tweets_rated["Entitle_t"] = entitle_pred_t
    tweets_rated["Neg_t"] = neg_pred_t
    tweets_rated["Blame_t"] = blame_pred_t
    tweets_rated["Misrep_em"] = misrep_pred_em
    tweets_rated["Entitle_em"] = entitle_pred_em
    tweets_rated["Neg_em"] = neg_pred_em
    tweets_rated["Blame_em"] = blame_pred_em
    tweets_rated["Misrep_s"] = misrep_pred_s
    tweets_rated["Entitle_s"] = entitle_pred_s
    tweets_rated["Neg_s"] = neg_pred_s
    tweets_rated["Blame_s"] = blame_pred_s
    tweets_rated["Polar_blob_t"] = polar_blob_t
    tweets_rated["Polar_blob_pos_t"] = polar_blob_pos_t
    tweets_rated["Subject_blob_t"] = subject_blob_t
    tweets_rated["Polar_blob_em"] = polar_blob_em
    tweets_rated["Polar_blob_pos_em"] = polar_blob_pos_em
    tweets_rated["Subject_blob_em"] = subject_blob_em
    tweets_rated["Polar_blob_s"] = polar_blob_s
    tweets_rated["Polar_blob_pos_s"] = polar_blob_pos_s
    tweets_rated["Subject_blob_s"] = subject_blob_s
    tweets_rated.to_csv(
        "/content/drive/MyDrive/Colab_Notebooks/PolitTweets/Classified/TwoMonthRatingsSpeechasTweetV"
        + v_num
        + j_name
        + end_date
        + "user"
        + str(j)
        + "V2.csv",
        index=False,
    )

In [ ]:
# Tabulate by politician, rank by negativity rating, export to file
ratings_df = pd.DataFrame()
ratings_df["Username"] = user_list
ratings_df["NumTweets"] = num_tweets
# pics_count is number of photos with text in them
# vids_count is number of videos with transcribed text
ratings_df["NumPics"] = pics_count
ratings_df["NumVids"] = vids_count
ratings_df["Misrep_t"] = misrep_pred_life_t
ratings_df["Entitle_t"] = entitle_pred_life_t
ratings_df["Neg_t"] = neg_pred_life_t
ratings_df["Blame_t"] = blame_pred_life_t
ratings_df["Misrep_em"] = misrep_pred_life_em
ratings_df["Entitle_em"] = entitle_pred_life_em
ratings_df["Neg_em"] = neg_pred_life_em
ratings_df["Blame_em"] = blame_pred_life_em
ratings_df["Misrep_s"] = misrep_pred_life_s
ratings_df["Entitle_s"] = entitle_pred_life_s
ratings_df["Neg_s"] = neg_pred_life_s
ratings_df["Blame_s"] = blame_pred_life_s
ratings_df["BlobPolar_t"] = polarity_blob_t
ratings_df["BlobPolarPos_t"] = polarity_blob_pos_t
ratings_df["BlobSubject_t"] = subjectivity_blob_t
ratings_df["BlobPolar_em"] = polarity_blob_em
ratings_df["BlobPolarPos_em"] = polarity_blob_pos_em
ratings_df["BlobSubject_em"] = subjectivity_blob_em
ratings_df["BlobPolar_s"] = polarity_blob_s
ratings_df["BlobPolarPos_s"] = polarity_blob_pos_s
ratings_df["BlobSubject_s"] = subjectivity_blob_s
ratings_df["User ID"] = polit_list_new
neg_pred_life_all = []
for i in range(len(neg_pred_life_t)):
    neg_pred_life_all.append(
        neg_pred_life_t[i] + neg_pred_life_em[i] + neg_pred_life_s[i]
    )
ratings_df["Neg"] = neg_pred_life_all
ratings_df.sort_values(
    by=["Neg"], axis=0, ascending=False, inplace=True, ignore_index=True
)
rank = []
for i in range(len(ratings_df)):
    rank.append(i + 1)
ratings_df["Rank"] = rank
ratings_df.to_csv(
    "/content/drive/MyDrive/Colab_Notebooks/PolitTweets/TwoMonthResults"
    + end_date
    + "/TwoMonthRatingsRawV"
    + v_num
    + end_date
    + "Day"
    + str(day_num)
    + ".csv",
    index=False,
)
ratings_df.head(20)